In [37]:
import random
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
os.environ["PATH"] += os.pathsep + "/opt/homebrew/bin"

In [39]:
from graphviz import Digraph

def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})

    for n in nodes:
        dot.node(name=str(id(n)), label = "{ data %.4f | grad %.4f }" % (n.data, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))

    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)

    return dot

In [25]:
class Value:
    """ stores a single scalar value and its gradient """

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op # the op that produced this node, for graphviz / debugging / etc

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward

        return out

    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward

        return out

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1.0
        for v in reversed(topo):
            v._backward()

    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

In [26]:
class Module:

    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0

    def parameters(self):
        return []

class Neuron(Module):

    def __init__(self, nin, nonlin=True):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(0)
        self.nonlin = nonlin

    def __call__(self, x):
        act = sum((wi*xi for wi,xi in zip(self.w, x)), self.b)
        return act.relu() if self.nonlin else act

    def parameters(self):
        return self.w + [self.b]

    def __repr__(self):
        return f"{'ReLU' if self.nonlin else 'Linear'}Neuron({len(self.w)})"

class Layer(Module):

    def __init__(self, nin, nout, **kwargs):
        self.neurons = [Neuron(nin, **kwargs) for _ in range(nout)]

    def __call__(self, x):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out) == 1 else out

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

    def __repr__(self):
        return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"

class MLP(Module):

    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1], nonlin=i!=len(nouts)-1) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

    def __repr__(self):
        return f"MLP of [{', '.join(str(layer) for layer in self.layers)}]"

In [55]:
mlp = MLP(4, [4, 4, 1])

xs = [
    [2.0, 3.0, -1.0, 9.0],
    [3.0, -1.0, 0.5, 2.0],
    [0.5, 1.0, 1.0, -2.0],
    [1.0, 1.0, -1.0, 0.9],
]
ys = [1.0, -1.0, -1.0, 1.0]

In [ ]:
res = mlp(xs[0])
draw_dot(res)

In [56]:
ypred = [mlp(x) for x in xs]
[print(yp.data) for yp in ypred]

for k in range(500):
    ypred = [mlp(x) for x in xs]
    loss = sum((ypred - actual)**2 for ypred, actual in zip(ypred, ys))
    mlp.zero_grad()
    loss.backward()
    print(f"{k}: {loss.data}")
    for p in mlp.parameters():
        p.data += -0.01 * p.grad

[yp.data for yp in ypred]

2.73919017137362
1.377188821545434
0.6353463216951118
0.5280530271389264
0: 11.572900682557632
1: 4.228321975201986
2: 4.105948904475435
3: 4.015476368502554
4: 3.9328009534895587
5: 3.850134092760261
6: 3.765817829342465
7: 3.675978836250803
8: 3.582327842344589
9: 3.4867924805999473
10: 3.385869413531102
11: 3.2104443050702187
12: 3.065681784964866
13: 2.9327329717360784
14: 2.806194939039785
15: 2.6839084201798045
16: 2.564649334119823
17: 2.447563405109781
18: 2.3269951037360777
19: 2.197287003641418
20: 2.077448815876745
21: 1.9657056709821157
22: 1.8599919148990125
23: 1.7615611315999142
24: 1.666720618547765
25: 1.6116734693548815
26: 1.715405910128017
27: 2.010556302718076
28: 3.4961478601612406
29: 1.894885218084398
30: 2.1853958384844083
31: 1.5277400192011146
32: 1.3959087960642358
33: 1.3263012483057048
34: 1.3679815624495637
35: 1.242255065478504
36: 1.3360937571067801
37: 1.171761558139867
38: 1.1071998964399128
39: 1.1031905585356072
40: 1.5541501243840514
41: 1.33352427

[1.000116795595575,
 -1.0048211353749326,
 -0.994478202132016,
 0.9999176451721699]